In [9]:
import sys
sys.path.append('/Users/tompease/Documents/Coding/airbnb')
from sklearn.linear_model import SGDRegressor, LinearRegression, ElasticNet, BayesianRidge
from sklearn.model_selection import ShuffleSplit
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate
from utils.data_loader import AirbnbLoader
import pandas as pd

loader = AirbnbLoader()
X, y = loader.load_airbnb('Price_Night', normalized=True)

cv_split = ShuffleSplit(n_splits = 5, test_size = .3, train_size = .7, random_state = 42)

In [10]:

MLA = [
  GradientBoostingRegressor(),
  LinearRegression(),
  SGDRegressor(),
  RandomForestRegressor(),
  ElasticNet(),
  SVR(),
  BayesianRidge(),
  KernelRidge(),
  XGBRegressor(),
  CatBoostRegressor(silent=True),
  DecisionTreeRegressor()
]


MLA_columns = ['MLA Name', 'MLA Parameters','Train MSE', 'Test MSE', 'Train r2', 'Test r2']
MLA_compare = pd.DataFrame(columns = MLA_columns)

row_index = 0

for alg in MLA:
  MLA_name = alg.__class__.__name__
  MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
  MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

  cv_results = cross_validate(alg, X=X, y=y, cv = cv_split , scoring=['r2', 'neg_mean_squared_error'], return_train_score=True) 

  MLA_compare.loc[row_index, 'Train MSE'] = cv_results['train_neg_mean_squared_error'].mean()
  MLA_compare.loc[row_index, 'Test MSE'] = cv_results['test_neg_mean_squared_error'].mean()
  MLA_compare.loc[row_index, 'Train r2'] = cv_results['train_r2'].mean()
  MLA_compare.loc[row_index, 'Test r2'] = cv_results['test_r2'].mean()

  row_index += 1

MLA_compare.sort_values(by = ['Test MSE'], ascending = False, inplace = True)

MLA_compare

,MLA Name,MLA Parameters,Train MSE,Test MSE,Train r2,Test r2
7,KernelRidge,"{'alpha': 1, 'coef0': 1, 'degree': 3, 'gamma':...",-0.008322,-0.008877,0.358231,0.327398
6,BayesianRidge,"{'alpha_1': 1e-06, 'alpha_2': 1e-06, 'alpha_in...",-0.008155,-0.008908,0.370992,0.322103
9,CatBoostRegressor,"{'loss_function': 'RMSE', 'silent': True}",-0.001313,-0.009871,0.898934,0.248969
3,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",-0.001337,-0.010411,0.89659,0.194651
0,GradientBoostingRegressor,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...",-0.003552,-0.011279,0.726178,0.126439
5,SVR,"{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.012361,-0.012756,0.047334,0.035863
8,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",-0.000109,-0.013032,0.991569,-0.021559
4,ElasticNet,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...",-0.012984,-0.013299,0.0,-0.002561
10,DecisionTreeRegressor,"{'ccp_alpha': 0.0, 'criterion': 'squared_error...",-0.000017,-0.019055,0.998675,-0.490723
1,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",-0.008113,-0.375994,0.374133,-32.620022
